```
This software is a part of GPU Ocean.

Copyright (C) 2019  SINTEF Digital

In this notebook we carry out prototyping for developing a new 
ensemble class that can be used for reading observations from file
and still work in the current Data Assimilation structure.

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Development of a new ensemble class based on files

In this notebook we carry out prototyping for developing a new 
ensemble class that can be used for reading observations from file
and still work in the current Data Assimilation structure.


## Set environment

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation, rc

import pycuda.driver as cuda
import os
import sys
import datetime

from importlib import reload
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../')))

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import IPythonMagic, CDKLM16, EnsembleFromFiles

from SWESimulators import BaseOceanStateEnsemble, SimReader, Observation
from SWESimulators import DataAssimilationUtils as dautils


In [2]:
%cuda_context_handler gpu_ctx

In [3]:
#Create output directory for images
#imgdir = 'double_jet'
#filename_prefix = imgdir + "/" + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S") + "_"
#os.makedirs(imgdir, exist_ok=True)
#print("Saving images to " + imgdir)

### Define functions for plotting

In [4]:
def imshow(im, interpolation="None", title=None, figsize=(4,4), interior=False):
    fig = plt.figure(figsize=figsize)
    
    if interior:
        im = plt.imshow(im[2:-2,2:-2], interpolation=interpolation, origin='lower')
    else:
        im = plt.imshow(im, interpolation=interpolation, origin='lower')
    
    plt.colorbar()
    if title is not None:
        plt.title(title)
        
def imshow3(eta, hu, hv, interpolation="None", title=None, figsize=(12,3), 
            interior=False, color_bar_from_zero=False):
    fig, axs = plt.subplots(1,3, figsize=figsize)
    
    eta_max = np.max(np.abs(eta))
    huv_max = max(np.max(np.abs(hu)), np.max(np.abs(hv)))
    eta_min = -eta_max
    huv_min = -huv_max
    if color_bar_from_zero:
        eta_min, huv_min = 0, 0
    
    if interior:
        eta_im = axs[0].imshow(eta[2:-2,2:-2], interpolation=interpolation, origin='lower', vmin=eta_min, vmax=eta_max)
    else:
        eta_im = axs[0].imshow(eta, interpolation=interpolation, origin='lower', vmin=eta_min, vmax=eta_max)
    axs[0].set_title("$\eta$")
    plt.colorbar(eta_im, ax=axs[0])
    
    if interior:
        hu_im = axs[1].imshow(hu[2:-2,2:-2], interpolation=interpolation, origin='lower', vmin=huv_min, vmax=huv_max)
    else:
        hu_im = axs[1].imshow(hu, interpolation=interpolation, origin='lower', vmin=huv_min, vmax=huv_max)
    axs[1].set_title("$hu$")
    plt.colorbar(hu_im, ax=axs[1])

    if interior:
        hv_im = axs[2].imshow(hv[2:-2,2:-2], interpolation=interpolation, origin='lower', vmin=huv_min, vmax=huv_max)
    else:
        hv_im = axs[2].imshow(hv, interpolation=interpolation, origin='lower', vmin=huv_min, vmax=huv_max)
    axs[2].set_title("$hv$")
    plt.colorbar(hv_im, ax=axs[2])

    if title is not None:
        plt.suptitle(title)
    plt.tight_layout()
    
def plotDrifters(observations, sim_reader, t, drifter_set=[]):
    drifter_positions = observations.get_drifter_position(t)

    fig = plt.figure(figsize=(7,7))
    ax = plt.subplot(111)
    nx, ny = sim_reader.get('nx'), sim_reader.get('ny')
    dx, dy = sim_reader.get('dx'), sim_reader.get('dy')

    emptyData =np.ma.masked_where(np.zeros((ny,nx)) > 1, np.zeros((ny,nx)))
    ax.imshow(emptyData, origin="lower", extent=[0, nx*dx, 0, ny*dy], cmap='binary')

    for i in range(drifter_positions.shape[0]):
        color = 'xkcd:pale cyan'
        if i in drifter_set:
            color = 'xkcd:tomato red'
        circ_end = matplotlib.patches.Circle((drifter_positions[i,0], drifter_positions[i,1]),
                                             3000, fill=True, zorder=10, color=color)
        ax.add_patch(circ_end)

    
def days_to_sec(days):
    return days*24*60*60

def truth_time_step(t):
    t = t - days_to_sec(3)
    return int(t/(60*60))

In [5]:
test = np.linalg.inv(np.diag([3,445]))
print(test)
print(type(test))
print(test.dtype)
test = test.astype(np.float32)
print(test.dtype)



[[0.33333333 0.        ]
 [0.         0.00224719]]
<class 'numpy.ndarray'>
float64
float32


# The new class

We base in on the OceanStateEnsemble.

In [125]:
if 'ensemble' in globals():
    ensemble.cleanUp()
    del ensemble
    
reload(BaseOceanStateEnsemble)
reload(Observation)
reload(EnsembleFromFiles)

ensemble_init_path = os.path.abspath('double_jet_ensemble_init/')
truth_path = os.path.abspath('double_jet_truth/')
#ensemble = EnsembleFromFile(

print(os.path.isdir(ensemble_init_path))
print(os.path.isdir(truth_path))

ensemble_nc_gen = (os.path.join(ensemble_init_path, file)  for file in os.listdir(ensemble_init_path) if file.endswith('.nc'))
ensemble_nc_files = list(ensemble_nc_gen)
print(type(ensemble_nc_files))
print(len(ensemble_nc_files))
print(ensemble_nc_files[10])
print(type(ensemble_nc_files[10]))
print()
print()

ensemble = EnsembleFromFiles.EnsembleFromFiles(gpu_ctx, 5, 
                                               ensemble_init_path, truth_path,
                                               0.1)
ensemble.configureObservations(drifterSet=[0,1,2], observationInterval=20)

True
True
<class 'list'>
100
/home/havahol/playground/gpu-ocean/havahol-gpu-ocean/gpu_ocean/demos/DAPaper/double_jet_ensemble_init/double_jet_case_55.nc
<class 'str'>


Welcome to the EnsembleFromFile
Ensemble directory:  /home/havahol/playground/gpu-ocean/havahol-gpu-ocean/gpu_ocean/demos/DAPaper/double_jet_ensemble_init
True state directory:  /home/havahol/playground/gpu-ocean/havahol-gpu-ocean/gpu_ocean/demos/DAPaper/double_jet_truth
self.write_netcdf_directory:  /home/havahol/playground/gpu-ocean/havahol-gpu-ocean/gpu_ocean/demos/DAPaper/ensemble_result_2019_04_26-18_20_35/


In [126]:
ensemble.observations.get_num_drifters()

3

In [127]:
test = [1,5,10]
test_array = np.arange(3,20)
print(type(test) is list, len(test))
print(test_array)
print(test_array[test])

True 3
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[ 4  8 13]


In [128]:
obs_time = (3*24+4)*60*60
depth = 230
obs = ensemble.observations.get_observation(obs_time, ensemble.mean_depth)


In [129]:
%%time
ensemble.stepToObservation(obs_time)

CPU times: user 3.6 s, sys: 564 ms, total: 4.17 s
Wall time: 3.98 s


In [130]:
print(obs)
print(ensemble.observeTrueState())
print(ensemble.observeTrueDrifters())

[[ 6.99850234e+04  3.94047109e+04  1.16094637e+01 -3.57905083e+01]
 [ 2.05943953e+05  4.14367539e+04 -3.70355644e+01 -5.55767775e+00]
 [ 3.49047875e+05  4.13093008e+04  3.60111656e+01 -8.43275738e+00]]
[[ 6.99850234e+04  3.94047109e+04  1.16094637e+01 -3.57905083e+01]
 [ 2.05943953e+05  4.14367539e+04 -3.70355644e+01 -5.55767775e+00]
 [ 3.49047875e+05  4.13093008e+04  3.60111656e+01 -8.43275738e+00]]
[[ 69985.0234375   39404.7109375 ]
 [205943.953125    41436.75390625]
 [349047.875       41309.30078125]]


In [112]:
print(obs)
print(ensemble.observeTrueState())
print(ensemble.observeTrueDrifters())

[[ 6.99850234e+04  3.94047109e+04  1.05356770e+01 -3.87975273e+01]
 [ 2.05943953e+05  4.14367539e+04 -3.60692711e+01 -4.89348936e+00]
 [ 3.49047875e+05  4.13093008e+04  3.68958321e+01 -7.73854160e+00]]
[[ 6.99850234e+04  3.94047109e+04  1.04338074e+01 -3.84223900e+01]
 [ 2.05943953e+05  4.14367539e+04 -3.57205162e+01 -4.84617376e+00]
 [ 3.49047875e+05  4.13093008e+04  3.65390854e+01 -7.66371679e+00]]
[[ 69985.0234375   39404.7109375 ]
 [205943.953125    41436.75390625]
 [349047.875       41309.30078125]]


In [122]:
ensemble.mean_depth

227.77611

In [63]:
ensemble.observations.get_observation_times()

array([ 259500.,  259800.,  260100., ..., 1122600., 1122900., 1123200.])